In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import datetime as dt
from dateutil.relativedelta import relativedelta


In [2]:
city_day_data = pd.read_csv('../data/comparison_city_day.csv')


In [3]:
def convert_string_to_date_time_obj(s):
    return dt.datetime.strptime(s,'%Y-%m-%d')

In [4]:
city_df = city_day_data.copy()
# city_df['Date'] = pd.to_datetime(city_df['Date'])
city_df.Date = city_df.Date.apply(lambda x : dt.datetime.strptime(x, '%Y-%m-%d'))
# city_df = city_df[city_df['AQI'].notna()].reset_index(drop=True)
city_df = city_df.sort_values(by = ['Date','City'])
city_df = city_df.reset_index(drop=True)
city_df.head(5)

,City,Date,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,Xylene,AQI,AQI_Bucket
0,Ahmedabad,2015-01-01,NaN,NaN,0.92,18.22,17.15,NaN,0.92,27.64,133.36,0.00,0.02,0.00,NaN,NaN
1,Bengaluru,2015-01-01,NaN,NaN,3.26,17.33,10.88,20.36,0.33,3.54,10.73,0.56,4.64,NaN,NaN,NaN
2,Chennai,2015-01-01,NaN,NaN,16.30,15.39,22.68,4.59,1.17,9.20,11.35,0.17,NaN,NaN,NaN,NaN
3,Delhi,2015-01-01,313.22,607.98,69.16,36.39,110.59,33.85,15.20,9.25,41.68,14.36,24.86,9.84,472.0,Severe
4,Lucknow,2015-01-01,NaN,NaN,2.11,13.46,4.57,NaN,12.15,169.57,25.92,1.35,3.93,NaN,NaN,NaN


In [5]:
cities = ['Delhi','Mumbai','Ahmedabad','Bengaluru','Kolkata']
city_df2 = city_df[city_df['Date'] >= '2019-03-01']
city_df2 = city_df2[city_df2['Date'] < '2020-06-01']
city_df2 = city_df2[city_df2['City'].isin(cities)]
city_df2 = city_df2.sort_values(by='Date')


SO2 = city_df2.loc[:,['City','Date','SO2']]
SO2 = SO2[SO2['SO2'].notna()].reset_index(drop=True)

NO2 = city_df2.loc[:,['City','Date','NO2']]
NO2 = NO2[NO2['NO2'].notna()].reset_index(drop=True)

PM25 = city_df2.loc[:,['City','Date','PM2.5']]
PM25 = PM25[PM25['PM2.5'].notna()].reset_index(drop=True)


In [24]:
PM10 = city_df2.loc[:,['City','Date','PM10']]
PM10.fillna(method='bfill',inplace=True)

PM10 = PM10.sort_values(by='Date')
PM10 = PM10.reset_index(drop=True)

i = 0

my_dict = {}
for row in PM10.itertuples():
    my_dict[(str(row[2].date()),row[1])] = row[3]

PM10['prev_data'] = PM10['PM10']
i = 0
for row in PM10.itertuples():
    if((row[2]) < convert_string_to_date_time_obj('2020-03-01')):
        i += 1
        continue
    prev_year = row[2] - relativedelta(years=1)
    prev_data = my_dict[(str(prev_year.date()),row[1])]
    PM10.at[i,'prev_data'] = prev_data
    i += 1

PM10['diff'] = -PM10['PM10'] + PM10['prev_data']

PM10 = PM10[PM10['Date'] >= '2020-03-01']
PM10 = PM10.pivot(index='Date',columns='City',values='diff')
print(PM10)


City        Ahmedabad  Bengaluru   Delhi  Kolkata  Mumbai
Date                                                     
2020-03-01       1.98      35.64   89.38   -65.78   29.48
2020-03-02     -54.54     -20.51  -20.97   -36.99   60.93
2020-03-03     -62.90     -61.59  -83.57    53.11  -78.39
2020-03-04     -24.92     -17.11  -12.95    83.93  -41.14
2020-03-05      -2.67       1.45   67.90    38.00   18.10
...               ...        ...     ...      ...     ...
2020-05-27      73.71       4.55   10.92    52.22   37.74
2020-05-28      70.61      29.23  -45.78    57.96   25.80
2020-05-29      59.66       8.91  136.09    48.46   29.46
2020-05-30      87.33      34.84  192.86    40.06    6.37
2020-05-31      68.42       9.23  297.34    39.19    7.61

[92 rows x 5 columns]


In [27]:
SO2 = city_df2.loc[:,['City','Date','SO2']]
SO2.fillna(method='bfill',inplace=True)

SO2 = SO2.sort_values(by='Date')
SO2 = SO2.reset_index(drop=True)

i = 0

my_dict = {}
for row in SO2.itertuples():
    my_dict[(str(row[2].date()),row[1])] = row[3]

SO2['prev_data'] = SO2['SO2']
i = 0
for row in SO2.itertuples():
    if((row[2]) < convert_string_to_date_time_obj('2020-03-01')):
        i += 1
        continue
    prev_year = row[2] - relativedelta(years=1)
    prev_data = my_dict[(str(prev_year.date()),row[1])]
    SO2.at[i,'prev_data'] = prev_data
    i += 1

SO2['diff'] = -SO2['SO2'] + SO2['prev_data']

SO2 = SO2[SO2['Date'] >= '2020-03-01']
SO2 = SO2.pivot(index='Date',columns='City',values='diff')
print(SO2)


City        Ahmedabad  Bengaluru  Delhi  Kolkata  Mumbai
Date                                                    
2020-03-01     -21.51      -2.16   3.62    -4.49    4.36
2020-03-02     -18.62      -1.79   2.15    -5.65   -1.89
2020-03-03     -70.07      -2.09  -2.00     5.68   -9.72
2020-03-04     -24.67      -1.51  -1.47    -1.60   -8.26
2020-03-05     -30.20      -3.01   2.94    -3.37  -11.04
...               ...        ...    ...      ...     ...
2020-05-27     137.27      -1.59   4.41    -3.44    8.47
2020-05-28     106.87      -2.17   8.91    -3.05    4.47
2020-05-29      76.27      -1.66  13.66    -3.54    3.75
2020-05-30     128.17      -2.31  12.04    -2.50    4.09
2020-05-31     138.54      -2.92  10.76    -3.14    6.84

[92 rows x 5 columns]


In [29]:
NO2 = city_df2.loc[:,['City','Date','NO2']]
NO2.fillna(method='bfill',inplace=True)

NO2 = NO2.sort_values(by='Date')
NO2 = NO2.reset_index(drop=True)

i = 0

my_dict = {}
for row in NO2.itertuples():
    my_dict[(str(row[2].date()),row[1])] = row[3]

NO2['prev_data'] = NO2['NO2']
i = 0
for row in NO2.itertuples():
    if((row[2]) < convert_string_to_date_time_obj('2020-03-01')):
        i += 1
        continue
    prev_year = row[2] - relativedelta(years=1)
    prev_data = my_dict[(str(prev_year.date()),row[1])]
    NO2.at[i,'prev_data'] = prev_data
    i += 1

NO2['diff'] = -NO2['NO2'] + NO2['prev_data']

NO2 = NO2[NO2['Date'] >= '2020-03-01']
NO2 = NO2.pivot(index='Date',columns='City',values='diff')
print(NO2)


City        Ahmedabad  Bengaluru  Delhi  Kolkata  Mumbai
Date                                                    
2020-03-01      25.03      22.73   8.14    13.54    6.55
2020-03-02       6.66      -2.64  11.82    31.77    6.89
2020-03-03     -11.92      -1.59  -5.99    80.23  -14.05
2020-03-04      11.88      -3.82  -5.52    41.32  -20.50
2020-03-05      43.13       2.58   7.08    22.88  -21.45
...               ...        ...    ...      ...     ...
2020-05-27      91.26      13.95  26.97     7.50    1.04
2020-05-28      84.21      10.46  44.35     2.32    2.37
2020-05-29      52.91       9.27  46.82     3.65    2.81
2020-05-30      74.56       4.66  53.29     5.44    3.06
2020-05-31      70.02      13.23  39.78     2.49    4.53

[92 rows x 5 columns]


In [32]:
PM25 = city_df2.loc[:,['City','Date','PM2.5']]
PM25.fillna(method='bfill',inplace=True)

PM25 = PM25.sort_values(by='Date')
PM25 = PM25.reset_index(drop=True)

i = 0

my_dict = {}
for row in PM25.itertuples():
    my_dict[(str(row[2].date()),row[1])] = row[3]

PM25['prev_data'] = PM25['PM2.5']
i = 0
for row in PM25.itertuples():
    if((row[2]) < convert_string_to_date_time_obj('2020-03-01')):
        i += 1
        continue
    prev_year = row[2] - relativedelta(years=1)
    prev_data = my_dict[(str(prev_year.date()),row[1])]
    PM25.at[i,'prev_data'] = prev_data
    i += 1

PM25['diff'] = -PM25['PM2.5'] + PM25['prev_data']

PM25 = PM25[PM25['Date'] >= '2020-03-01']
PM25 = PM25.pivot(index='Date',columns='City',values='diff')
print(PM25)


City        Ahmedabad  Bengaluru  Delhi  Kolkata  Mumbai
Date                                                    
2020-03-01     -10.95       1.98  51.00   -31.31   -2.27
2020-03-02      -9.85     -12.82  10.26   -11.60   -2.41
2020-03-03     -16.26     -33.80 -23.34    32.38  -27.83
2020-03-04      -0.32     -12.51  23.03    47.34  -19.91
2020-03-05      -4.52      -8.09  37.43    22.18  -10.44
...               ...        ...    ...      ...     ...
2020-05-27      26.22       5.96  21.44    33.44    6.41
2020-05-28      19.57      10.45  26.75    28.47    4.36
2020-05-29      18.11       5.50  50.18    26.73    7.60
2020-05-30      23.03      19.31  62.12    22.53    2.80
2020-05-31      23.68       6.97  67.06    17.50    4.93

[92 rows x 5 columns]


In [26]:
plot = make_subplots(
    rows=6, cols=1,
    subplot_titles=('Delhi','Mumbai','Ahmedabad','Bengaluru','Kolkata'))

plot.add_trace(go.Bar(x=PM10.index, y=PM10['Delhi'],
                    marker=dict(color=PM10['Delhi'],coloraxis="coloraxis")),1,1)

plot.add_trace(go.Bar(x=PM10.index, y=PM10['Mumbai'],
                    marker=dict(color=PM10['Mumbai'],coloraxis="coloraxis")),2,1)

plot.add_trace(go.Bar(x=PM10.index, y=PM10['Ahmedabad'],
                    marker=dict(color=PM10['Ahmedabad'],coloraxis="coloraxis")),3,1)

plot.add_trace(go.Bar(x=PM10.index, y=PM10['Bengaluru'],
                    marker=dict(color=PM10['Bengaluru'],coloraxis="coloraxis")),4,1)

plot.add_trace(go.Bar(x=PM10.index, y=PM10['Kolkata'],
                    marker=dict(color=PM10['Kolkata'],coloraxis="coloraxis")),5,1)


plot.update_layout(coloraxis=dict(colorscale='Temps'),showlegend=False,title_text="PM10")

plot.update_layout(plot_bgcolor='white')

plot.update_layout( width=800,height=1200,shapes=[
      dict(
      type= 'line',
#       yref= 'paper', y0= 0, y1= 1,
      xref= 'x', x0= '2020-03-25', x1= '2020-03-25'
    )
])


In [28]:
plot = make_subplots(
    rows=6, cols=1,
    subplot_titles=('Delhi','Mumbai','Ahmedabad','Bengaluru','Kolkata'))

plot.add_trace(go.Bar(x=SO2.index, y=SO2['Delhi'],
                    marker=dict(color=SO2['Delhi'],coloraxis="coloraxis")),1,1)

plot.add_trace(go.Bar(x=SO2.index, y=SO2['Mumbai'],
                    marker=dict(color=SO2['Mumbai'],coloraxis="coloraxis")),2,1)

plot.add_trace(go.Bar(x=SO2.index, y=SO2['Ahmedabad'],
                    marker=dict(color=SO2['Ahmedabad'],coloraxis="coloraxis")),3,1)

plot.add_trace(go.Bar(x=SO2.index, y=SO2['Bengaluru'],
                    marker=dict(color=SO2['Bengaluru'],coloraxis="coloraxis")),4,1)

plot.add_trace(go.Bar(x=SO2.index, y=SO2['Kolkata'],
                    marker=dict(color=SO2['Kolkata'],coloraxis="coloraxis")),5,1)


plot.update_layout(coloraxis=dict(colorscale='Temps'),showlegend=False,title_text="SO2")

plot.update_layout(plot_bgcolor='white')

plot.update_layout( width=800,height=1200,shapes=[
      dict(
      type= 'line',
#       yref= 'paper', y0= 0, y1= 1,
      xref= 'x', x0= '2020-03-25', x1= '2020-03-25'
    )
])


In [30]:
plot = make_subplots(
    rows=6, cols=1,
    subplot_titles=('Delhi','Mumbai','Ahmedabad','Bengaluru','Kolkata'))

plot.add_trace(go.Bar(x=NO2.index, y=NO2['Delhi'],
                    marker=dict(color=NO2['Delhi'],coloraxis="coloraxis")),1,1)

plot.add_trace(go.Bar(x=NO2.index, y=NO2['Mumbai'],
                    marker=dict(color=NO2['Mumbai'],coloraxis="coloraxis")),2,1)

plot.add_trace(go.Bar(x=NO2.index, y=NO2['Ahmedabad'],
                    marker=dict(color=NO2['Ahmedabad'],coloraxis="coloraxis")),3,1)

plot.add_trace(go.Bar(x=NO2.index, y=NO2['Bengaluru'],
                    marker=dict(color=NO2['Bengaluru'],coloraxis="coloraxis")),4,1)

plot.add_trace(go.Bar(x=NO2.index, y=NO2['Kolkata'],
                    marker=dict(color=NO2['Kolkata'],coloraxis="coloraxis")),5,1)


plot.update_layout(coloraxis=dict(colorscale='Temps'),showlegend=False,title_text="NO2")

plot.update_layout(plot_bgcolor='white')

plot.update_layout( width=800,height=1200,shapes=[
      dict(
      type= 'line',
#       yref= 'paper', y0= 0, y1= 1,
      xref= 'x', x0= '2020-03-25', x1= '2020-03-25'
    )
])


In [ ]:
plot = make_subplots(
    rows=6, cols=1,
    subplot_titles=('Delhi','Mumbai','Ahmedabad','Bengaluru','Kolkata'))

plot.add_trace(go.Bar(x=NO2.index, y=NO2['Delhi'],
                    marker=dict(color=NO2['Delhi'],coloraxis="coloraxis")),1,1)

plot.add_trace(go.Bar(x=NO2.index, y=NO2['Mumbai'],
                    marker=dict(color=NO2['Mumbai'],coloraxis="coloraxis")),2,1)

plot.add_trace(go.Bar(x=NO2.index, y=NO2['Ahmedabad'],
                    marker=dict(color=NO2['Ahmedabad'],coloraxis="coloraxis")),3,1)

plot.add_trace(go.Bar(x=NO2.index, y=NO2['Bengaluru'],
                    marker=dict(color=NO2['Bengaluru'],coloraxis="coloraxis")),4,1)

plot.add_trace(go.Bar(x=NO2.index, y=NO2['Kolkata'],
                    marker=dict(color=NO2['Kolkata'],coloraxis="coloraxis")),5,1)


plot.update_layout(coloraxis=dict(colorscale='Temps'),showlegend=False,title_text="NO2")

plot.update_layout(plot_bgcolor='white')

plot.update_layout( width=800,height=1200,shapes=[
      dict(
      type= 'line',
#       yref= 'paper', y0= 0, y1= 1,
      xref= 'x', x0= '2020-03-25', x1= '2020-03-25'
    )
])


In [33]:
plot = make_subplots(
    rows=6, cols=1,
    subplot_titles=('Delhi','Mumbai','Ahmedabad','Bengaluru','Kolkata'))

plot.add_trace(go.Bar(x=PM25.index, y=PM25['Delhi'],
                    marker=dict(color=PM25['Delhi'],coloraxis="coloraxis")),1,1)

plot.add_trace(go.Bar(x=PM25.index, y=PM25['Mumbai'],
                    marker=dict(color=PM25['Mumbai'],coloraxis="coloraxis")),2,1)

plot.add_trace(go.Bar(x=PM25.index, y=PM25['Ahmedabad'],
                    marker=dict(color=PM25['Ahmedabad'],coloraxis="coloraxis")),3,1)

plot.add_trace(go.Bar(x=PM25.index, y=PM25['Bengaluru'],
                    marker=dict(color=PM25['Bengaluru'],coloraxis="coloraxis")),4,1)

plot.add_trace(go.Bar(x=PM25.index, y=PM25['Kolkata'],
                    marker=dict(color=PM25['Kolkata'],coloraxis="coloraxis")),5,1)


plot.update_layout(coloraxis=dict(colorscale='Temps'),showlegend=False,title_text="PM25")

plot.update_layout(plot_bgcolor='white')

plot.update_layout( width=800,height=1200,shapes=[
      dict(
      type= 'line',
#       yref= 'paper', y0= 0, y1= 1,
      xref= 'x', x0= '2020-03-25', x1= '2020-03-25'
    )
])
